Module 1
 Analyzing the train data.f.

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("C:\\Users\\abhit\\OneDrive\\Desktop\\submissions\\train.csv")
print(train)

      case_id  Hospital_code Hospital_type_code  City_Code_Hospital  \
0           1              8                  c                   3   
1           2              2                  c                   5   
2           3             10                  e                   1   
3           4             26                  b                   2   
4           5             26                  b                   2   
...       ...            ...                ...                 ...   
4995     4996             19                  a                   7   
4996     4997             26                  b                   2   
4997     4998             32                  f                   9   
4998     4999             26                  b                   2   
4999     5000              9                  d                   5   

     Hospital_region_code  Available Extra Rooms in Hospital    Department  \
0                       Z                                  3  radioth

 Decoding the test data

In [4]:
test = pd.read_csv("C:\\Users\\abhit\\OneDrive\\Desktop\\submissions\\test.csv")
print(test)
test.dtypes

      case_id  Hospital_code Hospital_type_code  City_Code_Hospital  \
0      318439             21                  c                   3   
1      318440             29                  a                   4   
2      318441             26                  b                   2   
3      318442              6                  a                   6   
4      318443             28                  b                  11   
...       ...            ...                ...                 ...   
1995   320434             26                  b                   2   
1996   320435             26                  b                   2   
1997   320436             23                  a                   6   
1998   320437             25                  e                   1   
1999   320438             10                  e                   1   

     Hospital_region_code  Available Extra Rooms in Hospital  Department  \
0                       Z                                  3  gynecolog

case_id                                int64
Hospital_code                          int64
Hospital_type_code                    object
City_Code_Hospital                     int64
Hospital_region_code                  object
Available Extra Rooms in Hospital      int64
Department                            object
Ward_Type                             object
Ward_Facility_Code                    object
Bed Grade                            float64
patientid                              int64
City_Code_Patient                    float64
Type of Admission                     object
Severity of Illness                   object
Visitors with Patient                  int64
Age                                   object
Admission_Deposit                    float64
dtype: object

 Navigating the Missing Values in Train and Test

In [5]:
train.isnull().sum()

case_id                               0
Hospital_code                         0
Hospital_type_code                    0
City_Code_Hospital                    0
Hospital_region_code                  0
Available Extra Rooms in Hospital     0
Department                            0
Ward_Type                             0
Ward_Facility_Code                    0
Bed Grade                             2
patientid                             0
City_Code_Patient                    42
Type of Admission                     0
Severity of Illness                   0
Visitors with Patient                 0
Age                                   0
Admission_Deposit                     0
Stay                                  0
dtype: int64

In [6]:
test.isnull().sum()

case_id                               0
Hospital_code                         0
Hospital_type_code                    0
City_Code_Hospital                    0
Hospital_region_code                  0
Available Extra Rooms in Hospital     0
Department                            0
Ward_Type                             0
Ward_Facility_Code                    0
Bed Grade                             2
patientid                             0
City_Code_Patient                    21
Type of Admission                     0
Severity of Illness                   0
Visitors with Patient                 0
Age                                   0
Admission_Deposit                     0
dtype: int64

Data Healing

In [12]:
# Impute missing values in 'Bed Grade' with the mode
bed_grade_mode = train['Bed Grade'].mode()[0]
train['Bed Grade'].fillna(bed_grade_mode, inplace=True)

# Impute missing values in 'City_Code_Patient' with the mode
City_Code_Patient_mode = train['City_Code_Patient'].mode()[0]
train['City_Code_Patient'].fillna(City_Code_Patient_mode, inplace=True)

Data Completeness

In [16]:
# Impute missing values in 'Bed Grade' with the mode
bed_grade_mode = train['Bed Grade'].mode()[0]
test['Bed Grade'].fillna(bed_grade_mode, inplace=True)

# Impute missing values in 'City_Code_Patient' with the mode
city_code_patient_mode = train['City_Code_Patient'].mode()[0]
test['City_Code_Patient'].fillna(city_code_patient_mode, inplace=True)

 Transforming 'Stay' with LabelEncoder

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


test['Stay'] = -1

Charting the Unknown

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Use fit_transform to encode the values in the 'Stay' column
train['Stay'] = le.fit_transform(train['Stay'].astype('str'))



Data Convergence

In [20]:
df = pd.concat([train, test], ignore_index=True)

Transforming Categories into Numbers

In [21]:
# List of categorical column names
categorical_columns = ['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']

# Iterate through the list of categorical columns
for column in categorical_columns:
    le = LabelEncoder()  
    df[column] = le.fit_transform(df[column].astype(str))

Data Segmentation

In [22]:
train = df[df['Stay'] != -1]

Preparation for Prediction

In [23]:
test = df[df['Stay'] == -1]

Feature Engineering for Enhanced Predictive Analysis

In [24]:
import numpy as np
def get_countid_enocde(train, test, cols, name):
  temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  temp2 = test.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  train = pd.merge(train, temp, how='left', on= cols)
  test = pd.merge(test,temp2, how='left', on= cols)
  train[name] = train[name].astype('float')
  test[name] = test[name].astype('float')
  train[name].fillna(np.median(temp[name]), inplace = True)
  test[name].fillna(np.median(temp2[name]), inplace = True)
  return train, test




train, test = get_countid_enocde(train, test, ['patientid'], name = 'count_id_patient')
train, test = get_countid_enocde(train, test,
                                 ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
train, test = get_countid_enocde(train, test,
                                 ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')


# Create 'test1' DataFrame by dropping specified columns from 'test'
test1 = test.drop(columns=['Stay','patientid','Hospital_region_code','Ward_Facility_Code'], axis=1)
test1

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Available Extra Rooms in Hospital,Department,Ward_Type,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,count_id_patient,count_id_patient_hospitalCode,count_id_patient_wardfacilityCode
0,318439,21,2,3,3,2,3,2.0,2.0,0,2,2,7,3095.0,7.0,1.0,1.0
1,318440,29,0,4,2,2,3,2.0,2.0,1,2,4,7,4018.0,7.0,4.0,4.0
2,318441,26,1,2,3,2,1,4.0,2.0,0,2,3,7,4492.0,7.0,2.0,2.0
3,318442,6,0,6,3,2,1,2.0,2.0,1,2,3,7,4173.0,7.0,4.0,4.0
4,318443,28,1,11,2,2,2,2.0,2.0,1,2,4,7,4161.0,7.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,320434,26,1,2,1,1,2,2.0,8.0,1,0,4,5,8829.0,5.0,2.0,2.0
1996,320435,26,1,2,4,2,2,1.0,8.0,1,0,6,5,3507.0,5.0,2.0,2.0
1997,320436,23,0,6,3,2,1,1.0,8.0,1,0,3,5,4109.0,5.0,3.0,1.0
1998,320437,25,4,1,4,2,1,3.0,8.0,0,0,4,5,4155.0,5.0,3.0,2.0


Sculpting the Data

In [25]:

columns_to_drop_train = ['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code']

# Create 'train1' DataFrame by dropping specified columns from 'train'
train1 = train.drop(columns=columns_to_drop_train, axis=1)

Data Splitting for Model Mastery

In [38]:
from sklearn.model_selection import train_test_split
X1 = train1.drop(columns=['Stay'], axis=1)
y1 = train1['Stay']
# --- WRITE YOUR CODE FOR TASK 1 ---
X_train, X_test, y_train, y_test =  train_test_split(X1, y1, test_size=0.2, random_state=100)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (4000, 16)
X_test shape: (1000, 16)
y_train shape: (4000,)
y_test shape: (1000,)


The XGBoost Model Story

In [37]:
import xgboost
from sklearn.metrics import accuracy_score


classifier_xgb = xgboost.XGBClassifier(max_depth=4, learning_rate=0.1, n_estimators=800,
                                  objective='multi:softmax', reg_alpha=0.5, reg_lambda=1.5,
                                  booster='gbtree', n_jobs=4, min_child_weight=2, base_score= 0.75)

# --- WRITE YOUR CODE FOR TASK 2 ---
model_xgb = classifier_xgb.fit(X_train, y_train)

# Use the trained model to make predictions on the test data
prediction_xgb = model_xgb.predict(X_test)

# Calculate the accuracy of the XGBoost model's predictions
acc_score_xgb = accuracy_score(y_test, prediction_xgb)


print(acc_score_xgb)

0.396


The Final Predictive Model Transformation

In [36]:

label_mapping = {
     0: '0-10', 1: '11-20', 2: '21-30', 3: '31-40', 4: '41-50',
     5: '51-60', 6: '61-70', 7: '71-80', 8: '81-90', 9: '91-100',
     10: 'More than 100 Days'
 }

X_test1 = test1.drop(columns=['case_id'], axis=1)

# Use the trained 'classifier_xgb' model to make predictions on 'test1'
pred_xgb = classifier_xgb.predict(X_test1)

result_xgb = pd.DataFrame({'case_id': test1['case_id'], 'Stay': pred_xgb})

# Reorder the columns in 'result_xgb'
result_xgb = result_xgb[['case_id', 'Stay']]

# Replace numeric labels with provided categories in the 'Stay' column
result_xgb['Stay'] = result_xgb['Stay'].replace(label_mapping)
print(result_xgb)

      case_id   Stay
0      318439  11-20
1      318440  51-60
2      318441  11-20
3      318442  21-30
4      318443  51-60
...       ...    ...
1995   320434  51-60
1996   320435  71-80
1997   320436  21-30
1998   320437  11-20
1999   320438  21-30

[2000 rows x 2 columns]


Decoding Patient Stays

In [35]:
# Group the data by unique 'Stay' values and calculate the number of unique 'case_id' values
result = result_xgb.groupby('Stay')['case_id'].nunique()
print(result)

Stay
0-10                   60
11-20                 375
21-30                 869
31-40                 246
41-50                  32
51-60                 301
61-70                   7
71-80                  49
81-90                  27
91-100                  6
More than 100 Days     28
Name: case_id, dtype: int64
